In [1087]:
#Importing libraries

import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import math
from IPython.display import display

In [1088]:
#Defining function to get a dataframe from sql

def get_df_from_sql(SSH_required, query,key_path):

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [1089]:
#Getting the dataframe from sql query on ops main

SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'
query_path = 'pickup_query.sql'
with open(query_path,'r') as file:
    query = file.read()

df = get_df_from_sql(SSH_required, query, key_path)
# df = df[df['warehouse_id'] == 1514]

# print(df)

In [1090]:
#Making pickup slots on warehouses and pickup vehicle

df['pickuptime'] = pd.to_datetime(df['pickuptime'])

results = []

for (key, warehouse, pickup_vehicle), group in df.groupby(['key', 'warehouse_id', 'pickup_vehicle']):
    group = group.reset_index(drop=True)

    pickup_start_time = group.loc[0, 'pickuptime']

    for i in range(1, len(group)):
        current_time = group.loc[i, 'pickuptime']
        previous_time = group.loc[i - 1, 'pickuptime']
        
        # Check the difference between current and previous pickuptime
        if (current_time - previous_time).total_seconds() > 7200:  # More than two hours
            # End the current pickup slot and start a new one
            results.append({
                'key': key,
                'warehouse_id': warehouse,
                'pickup_vehicle': pickup_vehicle,
                'pickup_start_time': pickup_start_time,
                'pickup_end_time': previous_time
            })
            pickup_start_time = current_time  # Update the start time for the new slot
    
    # Append the last slot for the warehouse
    results.append({
        'key': key,
        'warehouse_id': warehouse,
        'pickup_vehicle': pickup_vehicle,
        'pickup_start_time': pickup_start_time,
        'pickup_end_time': group.loc[len(group) - 1, 'pickuptime']
    })

# Create DataFrame from results list
results_df = pd.DataFrame(results)

# Calculate slot_duration in minutes
results_df['slot_duration'] = ((results_df['pickup_end_time'] - results_df['pickup_start_time']).dt.total_seconds() / 60).apply(math.ceil)

# Sort DataFrame by warehouse_id and pickup_start_time (if not already sorted)
results_df.sort_values(by=['key', 'warehouse_id', 'pickup_start_time'], inplace=True)

# Calculate gaps between pickup slots in minutes
results_df['gap_to_next'] = results_df.groupby(['key', 'warehouse_id', 'pickup_vehicle'])['pickup_start_time'].shift(-1) - results_df['pickup_end_time']
results_df['gap_to_next'] = (results_df['gap_to_next'].dt.total_seconds() / 60)
results_df['gap_to_next'].fillna(0, inplace=True)
results_df['gap_to_next'] = (results_df['gap_to_next'].astype(int) + 1).replace(1, np.nan)

results_df.sort_values(by=['key', 'warehouse_id', 'pickup_vehicle'], inplace=True)

# print(results_df)

In [1091]:
# Adding actual load on slots

loads = []

for index, row in results_df.iterrows():
    key = row['key']
    warehouse_id = row['warehouse_id']
    pickup_vehicle = row['pickup_vehicle']
    slot_start_time = row['pickup_start_time']
    slot_end_time = row['pickup_end_time']
    
    load = 0  # Initialize load count for this slot

    filtered_df = df[(df['key'] == key) &
                     (df['warehouse_id'] == warehouse_id) &
                     (df['pickup_vehicle'] == pickup_vehicle) &
                     (df['pickuptime'] >= slot_start_time) &
                     (df['pickuptime'] <= slot_end_time)]

    # Count the number of AWBs within the slot time range and add to load
    load += len(filtered_df)
    
    # Append the total load count for this slot to the loads list
    loads.append(load)

# Assign the 'actual_load' column to results_df using the loads list
results_df['actual_load'] = loads
pickup_slots_df = results_df

# # Print or display the updated DataFrame with the 'actual_load' column
# print(pickup_slots_df)

# # Save DataFrame to Excel
# excel_file = 'pickup_slots.xlsx'
# pickup_slots_df.to_excel(excel_file, index=False)

# print(f"DataFrame saved to {excel_file}")


In [1092]:
#Loading pickup cutoff time data

query_path = 'pickup_cutoff_time.sql'
with open(query_path,'r') as file:
    query = file.read()

pickup_cutoff_df = get_df_from_sql(SSH_required, query, key_path)

pickup_cutoff_df['pickup_cutoff'] = pd.to_datetime(pickup_cutoff_df['pickup_cutoff'], format='%H:%M:%S', errors='coerce').dt.time
# pickup_cutoff_df = pickup_cutoff_df[pickup_cutoff_df['warehouse_id'] == 1514]
# display(pickup_cutoff_df)

In [1093]:
#Expanding cut off time data to a cut off datetime data with range of [d-3, d] days i.e. 4 days

# print(datetime.now())

# Step 1: Define the range of dates
start_date = datetime.now() - timedelta(days=3)  # Adjusted start date, datetime.now() is in IST already
end_date = datetime.now()

# Step 2: Create a new DataFrame to hold the expanded rows
expanded_rows = []

# Iterate through each row in pickup_cutoff_df
for index, row in pickup_cutoff_df.iterrows():
    key = row['key']
    warehouse_id = row['warehouse_id']
    pickup_cutoff = row['pickup_cutoff']

    # Iterate through each date in the range
    date_iterator = start_date
    while date_iterator <= end_date:
        # Combine current date with pickup_cutoff_time to create datetime
        pickup_datetime = datetime.combine(date_iterator.date(), pickup_cutoff)

        # Append row with key, pickup_cutoff_time, and current date
        expanded_rows.append({
            'key': key,
            'warehouse_id': warehouse_id,
            'pickup_cutoff': pickup_cutoff,
            'pickup_cutoff_time1': pickup_datetime
        })

        # Move to the next date
        date_iterator += timedelta(days=1)

# Create DataFrame from expanded rows
expanded_df = pd.DataFrame(expanded_rows)

merged_df = pd.merge(pickup_cutoff_df, expanded_df, left_on=['key', 'warehouse_id', 'pickup_cutoff'], right_on=['key', 'warehouse_id', 'pickup_cutoff'], how='left')

# Drop pickup_cutoff_time and rename pickup_cutoff_time1
merged_df.drop(columns=['pickup_cutoff'], inplace=True)
merged_df.rename(columns={'pickup_cutoff_time1': 'pickup_cutoff'}, inplace=True)
pickup_cutoff_df = merged_df

# Display the expanded DataFrame
# print(pickup_cutoff_df)

In [1094]:
#Matching pickups that happened with every possible pickup cutoff time

final_df = pd.merge(pickup_slots_df, pickup_cutoff_df, left_on = ['key', 'warehouse_id'], right_on = ['key', 'warehouse_id'], how='left')
# display(final_df)

In [1095]:
# There are some warehouse ids that are active but are not in pickup_cutoff_time, we are dropping them, but they need to be added in excel.pickup_cutoff_time

final_df.dropna(subset=['pickup_cutoff'], inplace=True)

In [1096]:
# Choosing rows with right pickup cutoff matching

final_df['slot_id'] = final_df['warehouse_id'].astype(str) + '-' + final_df['pickup_vehicle'].astype(str) + '-' + final_df['pickup_start_time'].astype(str)

# Calculate the duration and take the absolute value to see nearest cutoff time
final_df['start_to_pickup_cutoff'] = (final_df['pickup_start_time'] - final_df['pickup_cutoff']).abs()
final_df['end_to_pickup_cutoff'] = (final_df['pickup_end_time'] - final_df['pickup_cutoff']).abs()
# display(final_df)

# Function to apply the rules
def filter_rows(group):
    # Rule 1: If start_to_pickup_cutoff < 1 hour
    rule_1 = group[group['start_to_pickup_cutoff'] < pd.Timedelta(hours=1)]
    if not rule_1.empty:
        return rule_1.iloc[[0]]

    # Rule 2: If end_to_pickup_cutoff < 1 hour
    rule_2 = group[group['end_to_pickup_cutoff'] < pd.Timedelta(hours=1)]
    if not rule_2.empty:
        return rule_2.iloc[[0]]

    # Rule 3: If pickup_cutoff_time is within pickup_start_time and pickup_end_time
    rule_3 = group[(group['pickup_cutoff'] >= group['pickup_start_time']) & (group['pickup_cutoff'] <= group['pickup_end_time'])]
    if not rule_3.empty:
        return rule_3.iloc[[0]]

    # Rule 4: Closest pickup_cutoff_time < pickup_start_time
    group['diff'] = (group['pickup_start_time'] - group['pickup_cutoff']).abs()
    rule_4 = group[group['pickup_cutoff'] < group['pickup_start_time']]
    if not rule_4.empty:
        return rule_4.loc[[rule_4['diff'].idxmin()]]
    
    return pd.DataFrame()  # If no rule matches, return an empty DataFrame

# Apply the function to each group of slot_id
final_df = final_df.groupby('slot_id').apply(filter_rows).reset_index(drop=True)

# Drop the auxiliary column used for Rule 4
final_df.drop(columns=['diff'], inplace=True)

# display(final_df)



In [1097]:
# Cleaning data

final_df = final_df[['key', 'warehouse_id', 'pickup_vehicle', 'pickup_cutoff', 'pickup_start_time', 'pickup_end_time', 'actual_load']]
final_df = final_df.sort_values(by= ['key', 'pickup_cutoff'])
final_df = final_df.reset_index(drop=True)

summary_df = final_df.groupby(['key', 'warehouse_id', 'pickup_cutoff']).agg({
    'pickup_start_time': 'min',
    'pickup_end_time': 'max',
    'actual_load': 'sum'
}).reset_index()

# Display the summary DataFrame
# display(summary_df)

In [1098]:
# Ensures Missed Pickups are there

expanded_df.drop(columns=['pickup_cutoff'], inplace=True)
expanded_df.rename(columns={'pickup_cutoff_time1':'pickup_cutoff'}, inplace=True)
expanded_df.sort_values(by=['key', 'pickup_cutoff'], inplace=True)


# print(expanded_df)


In [1099]:
# Creating final table

final = pd.merge(expanded_df, summary_df, left_on=['key', 'warehouse_id', 'pickup_cutoff'], right_on=['key', 'warehouse_id', 'pickup_cutoff'], how='left')
# display(final)

In [1100]:
# Removing warehouses with no activity in all last 4 days

# Find all key values where pickup_start_time is NaT on all dates and remove them
key_with_all_NaT = final.groupby('key').filter(lambda x: x['pickup_start_time'].isna().all())['key'].unique()

# Convert the result to a list
key_with_all_NaT_list = key_with_all_NaT.tolist()

# Print the list
print(key_with_all_NaT_list)

final = final[~final['key'].isin(key_with_all_NaT_list)]
final['pickup_duration'] = final['pickup_end_time'] - final['pickup_start_time']
# display(final)

['100-1514-Bangalore', '101-732-Mumbai', '102-328-Mumbai', '185-359-Hyderabad', '204-815-Mumbai', '222-1394-Mumbai', '31-1534-Delhi', '31-1585-Bangalore', '31-1892-Mumbai', '31-65-Mumbai', '317-1626-Delhi', '327-182-Bangalore', '334-1425-Mumbai', '334-1491-Bangalore', '334-1504-Delhi', '340-1449-Delhi', '341-1453-Delhi', '359-1373-Delhi', '364-1558-Mumbai', '364-1590-Bangalore', '364-1590-Hyderabad', '364-1631-Hyderabad', '364-1721-Delhi', '364-1721-Jaipur', '364-1721-NCR', '374-1554-Delhi', '376-1623-Bangalore', '376-1661-Bangalore', '376-1670-Bangalore', '376-1672-Mumbai', '376-1710-Bangalore', '376-1879-Delhi', '376-1883-Delhi', '376-2001-Delhi', '377-1527-Hyderabad', '377-1571-Delhi', '378-1625-Mumbai', '378-1817-Delhi', '381-1614-Bangalore', '381-1639-Bangalore', '381-1669-Delhi', '381-1678-Mumbai', '381-1681-Hyderabad', '381-1687-Bangalore', '381-1688-Bangalore', '381-1723-Bangalore', '381-1758-Hyderabad', '381-1818-Hyderabad', '381-1853-Bangalore', '381-1952-Delhi', '381-1960-De

In [1101]:
# Creating status column

# Step 1: Check if 'pickup_status' column exists, if not, initialize it
if 'pickup_status' not in final.columns:
    final['pickup_status'] = np.nan

# Step 2: Preserve the 'Missed' status
missed_mask = final['pickup_status'] == 'Missed'

# Step 3: Define the conditions
conditions = [
    (final['pickup_start_time'].notna() & (final['pickup_start_time'] <= final['pickup_cutoff'] + pd.Timedelta(minutes=30))),
    (final['pickup_start_time'].notna() & (final['pickup_start_time'] > final['pickup_cutoff'] + pd.Timedelta(minutes=30))),
    (final['pickup_start_time'].isna() & (final['pickup_cutoff'] + pd.Timedelta(hours=6, minutes=30) < current_time)),
    (final['pickup_start_time'].isna() & (final['pickup_cutoff'] + pd.Timedelta(minutes=30) <= current_time)),
    (final['pickup_start_time'].isna() & (final['pickup_cutoff'] + pd.Timedelta(minutes=30) > current_time))
]

# Define the corresponding values for the conditions
choices = [
    'On-time pickup',
    'Late pickup',
    'Missed',
    'Delayed',
    'Pending'
]

# Step 4: Apply the conditions to update the 'pickup_status' column
final['pickup_status'] = np.select(conditions, choices, default='Unknown')

# Step 5: Reapply the 'Missed' status
final.loc[missed_mask, 'pickup_status'] = 'Missed'

# Display the final DataFrame
display(final)


,key,warehouse_id,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,pickup_duration,pickup_status
4,100-1514-Delhi,1514,2024-07-12 11:00:00,2024-07-12 11:12:55,2024-07-12 11:33:01,57.0,0 days 00:20:06,On-time pickup
5,100-1514-Delhi,1514,2024-07-12 17:30:00,NaT,NaT,NaN,NaT,Missed
6,100-1514-Delhi,1514,2024-07-13 11:00:00,NaT,NaT,NaN,NaT,Missed
7,100-1514-Delhi,1514,2024-07-13 17:30:00,2024-07-13 17:41:38,2024-07-13 17:44:01,7.0,0 days 00:02:23,On-time pickup
8,100-1514-Delhi,1514,2024-07-14 11:00:00,2024-07-14 11:20:50,2024-07-14 11:20:50,1.0,0 days 00:00:00,On-time pickup
...,...,...,...,...,...,...,...,...
923,87-1025-Delhi,1025,2024-07-15 10:30:00,2024-07-15 09:42:19,2024-07-15 09:42:19,1.0,0 days 00:00:00,On-time pickup
924,87-327-Mumbai,327,2024-07-12 16:00:00,2024-07-12 18:18:03,2024-07-12 18:18:55,10.0,0 days 00:00:52,Late pickup
925,87-327-Mumbai,327,2024-07-13 16:00:00,2024-07-13 19:01:38,2024-07-13 19:01:49,3.0,0 days 00:00:11,Late pickup
926,87-327-Mumbai,327,2024-07-14 16:00:00,2024-07-14 17:55:26,2024-07-14 17:55:36,4.0,0 days 00:00:10,Late pickup


In [1102]:
filepath = 'final_fm.xlsx'
final['pickup_duration'] = final['pickup_duration'].astype(str)
final.to_excel(filepath, index=False)